In [1]:
import numpy as np
from model import JGC, infer_GC_all_parallel
from evaluation import compute_score_metrics, compute_binary_metrics, compute_sign_sensitivity
import os
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #CPU

# Lorenz96 (F=10)

For Lorenz96 (F=10), we consider both GC _variable_ inference as well as time _lag_ inference. For time lag inference, the indices of the ground truth should be organized as [effect, time lag, cause] with the time lag organized as [$\eta,\eta-1,\eta-2,\ldots,3,2,1$].

In [2]:
lam = 1
data = np.load('data/Lorenz96 (F=10)/X_1.npy')
jgc = JGC(dim=data.shape[1],eta=5)
print('Data shape:',data.shape)
t1 = time.time()
var_binary_mat, var_score_mat, lag_binary_mat, lag_score_mat = jgc.infer_GC_all(data,lam,parallelize=True,
                                                                     processes=min(os.cpu_count()-1,data.shape[1]))
print('Time elapsed: %.1f seconds'%(time.time()-t1))

Data shape: (500, 20)
Time elapsed: 116.3 seconds


In [3]:
var_true = np.load('data/Lorenz96 (F=10)/GC.npy')
lag_true = np.load('data/Lorenz96 (F=10)/lags.npy')

var_auroc, var_auprc = compute_score_metrics(var_true,var_score_mat,exclude_self=False)
var_acc, var_ba, var_f = compute_binary_metrics(var_true,var_binary_mat,exclude_self=False)

lag_auroc, lag_auprc = compute_score_metrics(lag_true,lag_score_mat,exclude_self=False,lag=True)
lag_acc, lag_ba, lag_f = compute_binary_metrics(lag_true,lag_binary_mat,exclude_self=False,lag=True)

print('GC variables')
print('AUROC=%.4f, AUPRC=%.4f'%(var_auroc,var_auprc))
print('Acc=%.4f, Balanced Acc=%.4f, F-score=%.4f'%(var_acc, var_ba, var_f))
print('\nGC variables with time lag')
print('AUROC=%.4f, AUPRC=%.4f'%(lag_auroc,lag_auprc))
print('Acc=%.4f, Balanced Acc=%.4f, F-score=%.4f'%(lag_acc, lag_ba, lag_f))

GC variables
AUROC=0.9987, AUPRC=0.9948
Acc=0.9650, Balanced Acc=0.9641, F-score=0.9167

GC variables with time lag
AUROC=0.9998, AUPRC=0.9947
Acc=0.9925, Balanced Acc=0.9781, F-score=0.9112


# Lotka-Volterra System

For the Lotka-Volterra system, we consider only the GC variable inference and ignore the time lag. Here we will also consider the inference of interaction sign.

In [4]:
lam = 0.1
data = np.load('data/Lotka-Volterra/X_1.npy')
jgc = JGC(dim=data.shape[1],eta=1)
print('Data shape:',data.shape)
t1 = time.time()
var_binary_mat, var_score_mat, _, _ = jgc.infer_GC_all(data,lam,parallelize=True,processes=min(os.cpu_count()-1,data.shape[1]))
print('Time elapsed: %.1f seconds'%(time.time()-t1))

Data shape: (2000, 20)
Time elapsed: 355.6 seconds


In [5]:
var_true = np.load('data/Lotka-Volterra/GC.npy')

var_auroc, var_auprc = compute_score_metrics(var_true,var_score_mat,exclude_self=True)
var_acc, var_ba, var_f = compute_binary_metrics(var_true,var_binary_mat,exclude_self=True)

print('GC variables')
print('AUROC=%.4f, AUPRC=%.4f'%(var_auroc,var_auprc))
print('Acc=%.4f, Balanced Acc=%.4f, F-score=%.4f'%(var_acc, var_ba, var_f))

GC variables
AUROC=0.9999, AUPRC=0.9994
Acc=0.9921, Balanced Acc=0.9956, F-score=0.9639


Information on interaction sign can be extracted using `JGC.get_sign_all(threshold)`. When `threshold` is set to `True`, signs of all irrelevant variables are set to 0.

In [6]:
var_sign, lag_sign = jgc.get_sign_all(threshold=False)
sgc = np.load('data/Lotka-Volterra/signedGC.npy')

sens_plus, sens_neg = compute_sign_sensitivity(sgc,var_sign,exclude_self=True)
print('Sensitivity(+)=%.4f, Sensitivity(-)=%.4f'%(sens_plus,sens_neg))

Sensitivity(+)=1.0000, Sensitivity(-)=1.0000
